# HW 4 - Developing Neural Network Architectures

**Author**: Ewan Lister
Completed 05/08/2023

In this notebook we will use neural networks create fits for numeric data.

## I Reconsider the data from homework one:

    X=np.arange(0,31)
    Y=np.array([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41,
    40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53])

In [24]:
# imports
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso

X = torch.arange(0, 31, dtype=torch.float32).reshape(-1, 1)
Y = torch.tensor([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41,
                  40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53],
                 dtype=torch.float32).reshape(-1, 1)

data = dict(zip(X, Y))

print(data)


{tensor([0.]): tensor([30.]), tensor([1.]): tensor([35.]), tensor([2.]): tensor([33.]), tensor([3.]): tensor([32.]), tensor([4.]): tensor([34.]), tensor([5.]): tensor([37.]), tensor([6.]): tensor([39.]), tensor([7.]): tensor([38.]), tensor([8.]): tensor([36.]), tensor([9.]): tensor([36.]), tensor([10.]): tensor([37.]), tensor([11.]): tensor([39.]), tensor([12.]): tensor([42.]), tensor([13.]): tensor([45.]), tensor([14.]): tensor([45.]), tensor([15.]): tensor([41.]), tensor([16.]): tensor([40.]), tensor([17.]): tensor([39.]), tensor([18.]): tensor([42.]), tensor([19.]): tensor([44.]), tensor([20.]): tensor([47.]), tensor([21.]): tensor([49.]), tensor([22.]): tensor([50.]), tensor([23.]): tensor([49.]), tensor([24.]): tensor([46.]), tensor([25.]): tensor([48.]), tensor([26.]): tensor([50.]), tensor([27.]): tensor([53.]), tensor([28.]): tensor([55.]), tensor([29.]): tensor([54.]), tensor([30.]): tensor([53.])}


### (i) Fit the data to a three layer feed forward neural network.

In [54]:
# Define the neural network architecture
class ThreeLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 10)  # input layer -> hidden layer
        self.fc2 = nn.Linear(10, 10) # hidden layer -> hidden layer
        self.fc3 = nn.Linear(10, 1)  # hidden layer -> output layer
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# initialize network
net = ThreeLayerNet()

# use SGD for fitting
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

# loss function
criterion = nn.MSELoss()

# create training data loader
train_loader = torch.utils.data.DataLoader(dataset=data, batch_size=1, shuffle=True)


# Train the neural network using gradient descent
num_epochs = 15
for epoch in range(num_epochs):
    for i, (x) in enumerate(X):
        optimizer.zero_grad()
        outputs = net(x)
        loss = criterion(outputs, Y[i])
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 31 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, 31, loss.item()))



Epoch [1/15], Step [31/31], Loss: 2566.2896
Epoch [2/15], Step [31/31], Loss: 2253.1060
Epoch [3/15], Step [31/31], Loss: 842.2803
Epoch [4/15], Step [31/31], Loss: 15.4006
Epoch [5/15], Step [31/31], Loss: 12.0758
Epoch [6/15], Step [31/31], Loss: 12.0206
Epoch [7/15], Step [31/31], Loss: 9.8867
Epoch [8/15], Step [31/31], Loss: 10.8064
Epoch [9/15], Step [31/31], Loss: 11.9087
Epoch [10/15], Step [31/31], Loss: 11.8223
Epoch [11/15], Step [31/31], Loss: 9.7253
Epoch [12/15], Step [31/31], Loss: 10.2602
Epoch [13/15], Step [31/31], Loss: 9.8915
Epoch [14/15], Step [31/31], Loss: 10.5323
Epoch [15/15], Step [31/31], Loss: 10.3956


### (ii) Using the first 20 data points as training data, fit the neural network. Compute the least-square error for each of these over the training points. Then compute the least square error of these models on the test data which are the remaining 10 data points.

In [55]:
# isolate first 20 data points
x_train = X[0:20]
y_train = Y[0:20]
x_test = X[20:31]
y_test = Y[20:31]

# train network on first 20 data points, examine progress of SGD via print statements
num_epochs = 15
for epoch in range(num_epochs):
    for i, (x) in enumerate(x_train):
        optimizer.zero_grad()
        outputs = net(x)
        loss = criterion(outputs, y_train[i])
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 31 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, 31, loss.item()))

for i, (x) in enumerate(x_train):
    outputs = net(x)
    error = criterion(outputs, y_train[i])
    print('Train error for x = {}, y = {} : {:.4f}'.format(x, y_train[i], error))

for i, (x) in enumerate(x_test):
    outputs = net(x)
    error = criterion(outputs, y_test[i])
    print('Test error for x = {}, y = {} : {:.4f}'.format(x, y_test[i], error))

Error for x = tensor([0.]), y = tensor([30.]) : 0.0065
Error for x = tensor([1.]), y = tensor([35.]) : 5.6075
Error for x = tensor([2.]), y = tensor([33.]) : 0.2177
Error for x = tensor([3.]), y = tensor([32.]) : 6.3447
Error for x = tensor([4.]), y = tensor([34.]) : 2.4687
Error for x = tensor([5.]), y = tensor([37.]) : 0.1417
Error for x = tensor([6.]), y = tensor([39.]) : 1.7534
Error for x = tensor([7.]), y = tensor([38.]) : 0.5303
Error for x = tensor([8.]), y = tensor([36.]) : 14.2900
Error for x = tensor([9.]), y = tensor([36.]) : 23.3344
Error for x = tensor([10.]), y = tensor([37.]) : 23.2196
Error for x = tensor([11.]), y = tensor([39.]) : 7.9449
Error for x = tensor([12.]), y = tensor([42.]) : 0.0329
Error for x = tensor([13.]), y = tensor([45.]) : 10.1209
Error for x = tensor([14.]), y = tensor([45.]) : 10.1209
Error for x = tensor([15.]), y = tensor([41.]) : 0.6702
Error for x = tensor([16.]), y = tensor([40.]) : 3.3076
Error for x = tensor([17.]), y = tensor([39.]) : 7.94

### (iii) Repeat (iii) but use the first 10 and last 10 data points as training data. Then fit the model to the test data (which are the 10 held out middle data points). Compare these results to (iii)




### (iv) Compare the models fit in homework one to the neural networks in (ii) and (iii)

## II Now train a feedforward neural network on the MNIST data set. You will start by performing the following analysis:

### (i) Compute the first 20 PCA modes of the digit images.

### (ii) Build a feed-forward neural network to classify the digits. Compare the results of the neural network against LSTM, SVM (support vector machines) and decision tree classifiers.